<a href="https://colab.research.google.com/github/Chigaga/area_calculation/blob/main/area_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sunpy.map
from PIL import Image
from math import asin, sqrt, pi

In [ ]:

def masking_nan(map_img, segmented_img):
    """
    Masks pixels in a map image where the corresponding pixel in a segmented image
    is not useful (255) by setting them to NaN.

    Parameters:
    map_img (sunpy.map.Map): The map image to be masked.
    segmented_img (sunpy.map.Map): The segmented image.

    Returns:
    sunpy.map.Map: The masked map image.
    """
    mask = map_img.data
    mask[segmented_img.data < 255] = np.nan
    masked_img = sunpy.map.Map(mask, map_img.meta)
    return masked_img


def rad_parallel_cut(rad):
    """
    Computes the radiuses of circles at the Sun parallels given a radius in pixels.

    Parameters:
    rad (int): The radius in pixels.

    Returns:
    numpy.ndarray: An array containing the computed radiuses of circles.
    """
    out = np.zeros(2 * rad)
    for it in range(rad):
        out[it] = int(np.sqrt((rad ** 2 - (rad - it - 1) ** 2)))
        out[2 * rad - it - 1] = out[it]
    return out

In [ ]:
def full_disk_mask(segmented_img):
    """
    Creates a full-disk mask by pasting a cropped segmented image onto a blank image.

    Parameters:
    segmented_img (sunpy.map.Map): The cropped segmented image.

    Returns:
    sunpy.map.Map: The full-disk mask image of 2048x2048 px size
    """
    data_full_img = np.arange(0, 2048 * 2048).reshape(2048, 2048) * np.nan
    header = segmented_img.meta
    BD_full_img = sunpy.map.Map(data_full_img, header)
    BD_full_img.meta['crpix1'] = 1024
    BD_full_img.meta['crpix2'] = 1024

    final_size = BD_full_img.data.shape  # [rows, columns]
    crpix_x_full = round(BD_full_img.meta['crpix1'])
    crpix_y_full = round(BD_full_img.meta['crpix2'])
    crpix_x_crop = round(segmented_img.meta['crpix1'])
    crpix_y_crop = round(segmented_img.meta['crpix2'])
    xpix = crpix_x_full - crpix_x_crop  # coordinate of the bottom left pixel
    ypix = crpix_y_full - crpix_y_crop

    # Create final mask
    data = segmented_img.data
    height, length = data.shape
    fulldisk_mask = np.zeros(final_size) * np.nan

    # Using PIL library
    image_full = Image.fromarray(fulldisk_mask)
    image_crop = Image.fromarray(data)
    box = (xpix, ypix)  # Note: Image coordinate system starts from the upper left corner
    Image.Image.paste(image_full, image_crop, box=box)
    fulldisk_data = np.array(image_full)
    fulldisk_mask = sunpy.map.Map(fulldisk_data, BD_full_img.meta)

    return fulldisk_mask

In [ ]:
def find_area(masked_map):
    
    # Get useful information from the masked map
    R_pix = masked_map.meta['rsun_obs'] / masked_map.meta['cdelt1'] # Sun radius in pixels
    R_km = masked_map.meta['rsun_ref'] * 10**(-3) # Sun radius in km
    cenX, cenY = int(masked_map.meta['crpix1']), int(masked_map.meta['crpix2']) # Reference pixel coordinates

    # Calculate the radius of the parallel cut
    rad = rad_parallel_cut(R_pix)

    nanleft = cenX - rad  # Distance from the left (grid start)
    dist_top = cenY - rad  # Distance from the top (grid start) to the Sun from the top 

    dimming_pos = np.argwhere(~np.isnan(masked_map.data))  # Array of non-NaN pixels
    i_all, j_all = dimming_pos[:, 0], dimming_pos[:, 1]  # Coordinates of non-NaN pixels

    pix_on_left = 0
    Area_pixel = []
    
    for it in range(len(i_all)):
        i = i_all[it] - dist_top  # Y coordinate (rows)
        j = j_all[it] - nanleft[rad]  # X coordinate (columns)

        if j < 1:  # Skip if on the left from CenX
            continue
        
        if j_all[it] >= cenX:  # Pixels on the left have the same area
            j = j_all[it] - cenX
            j = rad - j
            pix_on_left += 1
            if j < 1:
                continue
        
        if i_all[it] > cenY:  # If on the bottom from CenY
            i = i_all[it] - cenY
            i = rad - i
        
        # Calculate the area of a pixel
        area_sun_km = 2 * pi * R_km**2
        area_sun_pix = pi * rad**2
        area_pixel = area_sun_km / area_sun_pix
        
        # Calculate the area of the Sun as a sphere, having the projection to a circle
        R = R_km
        m = rad
        a = R / m
        dt = -R + j * a
        gm = (m - i) * a
        area_sphere = R * (asin((gm + a) / sqrt(R**2 - j**2 * a**2)) - asin(gm / sqrt(R**2 - j**2 * a**2)))
        Area_pixel.append(area_sphere / area_pixel)
    
    # Calculate the total area of the dimming region in km^2
    area = np.nansum(Area_pixel) * area_pixel
    
    return area

In [ ]:
def find_area(x,y,masked_map):
  #add the checking of that the pixels is located ondisk
    
    R_pix= masked_map.meta['rsun_obs']/masked_map.meta['cdelt1'] # sun radius in pixels
    rad=int(R_pix)
    R_km= masked_map.meta['rsun_ref']*10**(-3) # sun radius in km
    
    ## Useful information
    
    # Note that CRPIXn follows the Fortran convention of counting from 1 to N,
    #instead of from 0 to N − 1 as is done in some programming languages.
    cenX= int(masked_map.meta['crpix1']) # X axis reference pixel 
    cenY= int(masked_map.meta['crpix2']) # Y axis reference pixel

    rad_circle= rad_parallel_cut(rad)

    nanleft=np.array(cenX-rad_circle) # Distance from the left (grid start)
        
    dist_top= cenY-rad # Distance from the top (grid start) to the Sun from the top 
    
    dimming_pos= np.argwhere(masked_map.data==masked_map.data) # output in ascending order (y from top to bottom, x from left to right)
    
    # y coordinate (rows)
    i_all = y + 1
    # x coordinate (columns)
    j_all = x + 1

    # Note! cenX and cenY follow the Fortran convention.
    # The same should be valid for i and j.

    Area_pixel=[]
    
  
    i= i_all-dist_top # coordinate i along Y
    j= j_all-nanleft[rad]   
    if (j<1):
        continue
    
    # if on the left from CenX (pixels on the same distance have the same area)
    if (j_all>=cenX):
        j=(j_all-cenX) 
        j=rad-j  

        if (j<1):
            continue
    
    # if on the bottom from CenY
    if (i_all > cenY):
        i = i_all-cenY
        i = rad-i
    

    
    R = R_km # radius of Sun in km
    m = rad
    a = R/m # ratio of radius in km to radius in pixels
    dt = -R+ j*a
    gm = (m-i)*a
    
    # Calculate the integral by summing up (integration)
    SS=0
    step=1000
    dx= a/step # Divide integration interval by 1000 (very small step)
    for k in range(step):
        x=a*(-rad+j-0.5)+(k-1)*dx
        
        #limits of integral over Y axis 
        int_up=a*(m-i+0.5)  
        int_down=a*(m-i-0.5)                     
                  

        dSS=R*(asin(int_up/sqrt(R**2-(x+dx/2)**2))-asin(int_down/sqrt(R**2-(x+dx/2)**2 ))) * dx

        Area_pixel.append(SS)

  A = sum(Area_pixel)
  return A